In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DetectionTest

/content/drive/MyDrive/DetectionTest


In [ ]:
pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!pip install spaces

In [ ]:
import numpy as np
import csv, time

In [ ]:
import gradio as gr
import smtplib
from email.mime.text import MIMEText
import PIL.Image as Image
import cv2
from ultralytics import YOLO
from ultralytics import solutions
import tempfile
import os
import time
from moviepy.editor import ImageSequenceClip

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


  if event.key is 'enter':



# Test 1

In [ ]:
import gradio as gr
import cv2
import pandas as pd
import numpy as np
from ultralytics import solutions
from moviepy.editor import ImageSequenceClip
from datetime import datetime, timedelta

# Định nghĩa vùng, màu sắc, lớp
region_points = [[220, 891], [668, 573], [1453, 563], [1832, 886]]
color_purple_bgr = (255, 153, 204)
alpha = 0.3
class_names = ["car", "motorbike", "truck", "bus"]

def process_video(input_video, start_date):
    # 1. Chuyển start_date từ string sang datetime
    start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")

    # 2. Mở video
    cap = cv2.VideoCapture(input_video)
    assert cap.isOpened(), "Could not read the input video"

    # 3. Lấy thông số video
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 4. Khởi tạo RegionCounter của ultralytics
    regioncounter = solutions.RegionCounter(
        show=False,
        region=region_points,
        model="/content/drive/MyDrive/DetectionTest/best.pt",
        tracker='botsort.yaml',
        conf=0.3,
        iou=0.5,
        show_labels=True,
        show_in=True,
        device=0
    )

    # 5. Chuyển region_points thành định dạng cho OpenCV
    pts = np.array(region_points, np.int32).reshape((-1, 1, 2))

    # 6. Cấu hình lấy mẫu: mỗi 10 giây
    sample_interval_sec = 10
    interval = int(fps * sample_interval_sec)

    frames = []
    data = []
    frame_id = 0
    sample_idx = 0  # Đếm số lần mẫu (phút thứ N)

    # 7. Vòng xử lý từng frame
    while True:
        success, im0 = cap.read()
        if not success:
            break

        # 7.1. Phân tích bằng RegionCounter
        results = regioncounter(im0)
        frame = results.plot_im.copy()
        overlay = frame.copy()

        # 7.2. Vẽ đa giác vùng đếm
        cv2.fillPoly(overlay, [pts], color_purple_bgr)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        # 7.3. Lấy mẫu mỗi interval frame, bỏ frame_id == 0
        if frame_id != 0 and frame_id % interval == 0:
            sample_idx += 1
            actual_time = start_date + timedelta(minutes=sample_idx)

            # Đếm tổng xe trong vùng
            total_in_region = 0
            for box, cls in zip(regioncounter.boxes, regioncounter.clss):
                x1, y1, x2, y2 = box
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)
                if cv2.pointPolygonTest(pts, (cx, cy), False) >= 0:
                    total_in_region += 1

            # Lưu mẫu
            data.append([actual_time, total_in_region])

        # Lưu frame để tạo video output
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_id += 1

    cap.release()

    # 8. Tạo DataFrame và ghi CSV
    df = pd.DataFrame(data, columns=["datetime", "total_in_region"])
    # Định dạng datetime thành chuỗi để hiển thị đủ phút và luôn có :00 giây
    df["datetime"] = df["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S")

    csv_path = "/content/drive/MyDrive/DetectionTest/csv/vehicle_region_count.csv"
    df.to_csv(csv_path, index=False)

    # 9. Tạo video output lại từ frames
    output_video_path = "region_output.mp4"
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(
        output_video_path,
        codec="libx264",
        audio=False,
        verbose=False,
        logger=None
    )

    return output_video_path

# 10. Giao diện Gradio
demo = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Video(label="Upload Video"),
        gr.Textbox(label="Start Date (YYYY-MM-DD HH:MM:SS)",
                   value="2025-06-20 00:00:00")
    ],
    outputs=gr.Video(label="Processed Output"),
    title="Vehicle Region Counter",
    description="Upload a video and get a processed output with region-based vehicle counting per minute."
)

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7df27e3a5e9710ac2c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
